In [100]:
%pylab inline
import rlcompleter, readline
readline.parse_and_bind('tab: complete')
#import sys
#reload(sys)
#sys.setdefaultencoding('utf8')

In [101]:
import csv
amazon_data = []
with open("sentiment_labelled_sentences/amazon_cells_labelled.txt") as f:
    for line in f:
        amazon_data.append(line.strip('\n').split('\t'))
amazon_data = np.asarray(amazon_data)

imdb_data = []
with open("sentiment_labelled_sentences/imdb_labelled.txt") as f:
    for line in f:
        imdb_data.append(line.strip('\n').split('\t'))
imdb_data = np.asarray(imdb_data)

yelp_data = []
with open("sentiment_labelled_sentences/yelp_labelled.txt") as f:
    for line in f:
        yelp_data.append(line.strip('\n').split('\t'))
yelp_data = np.asarray(yelp_data)

In [102]:
def parse_data(data):
    sentence = []
    score = []
    line = ""
#    print type(line)
    for row in data:
#        print type(row[0])
#        line = line + row[0]
#        print line
        if row[1] !='':
            sentence.append(row[0])
            score.append(int(row[1]))
#            line = ""
    return [sentence,score]

#Preprocessing

In [399]:
#parse data
[amazon_sentence,amazon_score] = parse_data(amazon_data)
[imdb_sentence,imdb_score] = parse_data(imdb_data)
[yelp_sentence,yelp_score] = parse_data(yelp_data)
#len([i for i in amazon_score if i==1])
#len([i for i in imdb_score if i==1])
len([i for i in yelp_score if i==1])
# the labels 0 and 1 is balanced

500

In [104]:
#lowercase 
#lowercase text data in case of program misunderstand same words as different words.
amazon_sentence = np.char.lower(amazon_sentence)
imdb_sentence = np.char.lower(imdb_sentence)
yelp_sentence = np.char.lower(yelp_sentence)

In [105]:
#tokenize
#segment sentence into words.
from nltk import word_tokenize
def tokenizer(data):
    token_data = []
    for line in data:
        line = word_tokenize(line)
        token_data.append(line)
    return np.asarray(token_data)

In [106]:
a_token = tokenizer(amazon_sentence)
i_token = tokenizer(imdb_sentence)
y_token = tokenizer(yelp_sentence)

In [107]:
#strip punctuation
#strip punctuation because punctuation dosen't make contribution to clustering
import string
def strip_punctuation(data):
    punctuation = list(string.punctuation)
    punctuation.extend(['``','""','...'])
#    print punctuation
#    for each in punctuation:
#        index = np.where(data == 'so')
#    return index
#    print data.shape
    strip = []
    for line in data:
        tmp = np.in1d(line,punctuation)
        index = np.where(tmp == True)
        strip_line = np.delete(line,index)
        strip.append(strip_line)
    return np.asarray(strip)

In [108]:
a_strip = strip_punctuation(a_token)
i_strip = strip_punctuation(i_token)
y_strip = strip_punctuation(y_token)

In [109]:
#stopwords
#same reason with punctuation
from nltk.corpus import stopwords
sw = stopwords.words('english')
sw.extend(['\xc2\x96'])
print sw
#print sw
def remove_stopwords(data):
    rm_data = []
    for line in data:
        tmp = np.in1d(line,sw)
#        print tmp
        index = np.where(tmp == True)
        rm_line = np.delete(line,index)
        rm_data.append(rm_line)
#        print rm_line
    return rm_data

In [110]:
a_rm = remove_stopwords(a_strip)
#np.delete(i_strip,np.where(i_strip == '\xc2\x96'.decode('utf-8')))
i_rm = remove_stopwords(i_strip)
y_rm = remove_stopwords(y_strip)
#print i_rm

In [111]:
#lemmatization
#words with different type but mean same thing
from nltk.stem import WordNetLemmatizer
def lemmatizer(data):
    l = WordNetLemmatizer()
    lem_data = []
    lem_line = []
    for line in data:
        for item in line:
            lem_line.append(l.lemmatize(item))
        print lem_line
        lem_data.append(lem_line)
        lem_line = []
    return np.asarray(lem_data)

In [112]:
a_lem = lemmatizer(a_rm)
i_lem = lemmatizer(i_rm)
y_lem = lemmatizer(y_rm)

In [133]:
reload(sys)
#split training and testing set
def trainingAndTesting(sentence,score):
    score = np.asarray(score)
    positive = sentence[np.where(score == 1)]
    negative = sentence[np.where(score == 0)]
    train = np.concatenate((positive[:1200],negative[:1200]),axis=0)
    train_label = [1]*1200+[0]*1200
    test = np.concatenate((positive[1200:],negative[1200:]),axis=0)
    return train,test,train_label

In [137]:
sentence = np.concatenate((a_lem,i_lem,y_lem),axis=0)
score = amazon_score+imdb_score+yelp_score
train,test,train_label = trainingAndTesting(sentence,score)

In [368]:
np.where(dic == 'packed')

(array([], dtype=int64),)

#Bag of word

In [160]:
# build up dict through training set
def build_dic(train):
    dic = []
    for line in train:
        for word in line:
            dic.append(word)
    dic = list(set(dic))
    return dic
dic = build_dic(train)
len(dic)

4361

In [161]:
#build up feature vector
#testing set might include data that doen't occur in training data. That would influence the performance of training.
def build_vector(train):
    vectors = []
    for line in train:
        vector = [0]*len(dic)
        for word in line:
            if word in dic:
                vector[dic.index(word)] += 1
        vectors.append(vector)
    return vectors

In [163]:
train_vectors = build_vector(train)

In [169]:
test_vectors = build_vector(test)

In [171]:
#report any two feature vectors of reviews in training set
#train_vectors[50]

In [362]:
np.savetxt('label.txt',train_label)

In [386]:
np.savetxt('train_feature_vectors.txt',train_vectors)
np.savetxt('test_feature_vectors.txt',test_vectors)

In [393]:
[i for i in range(len(train_vectors)) if np.sum(train_vectors[i]) == 0]
[i for i in range(len(test_vectors)) if np.sum(test_vectors[i]) == 0]

[405, 417, 455]

In [385]:
import csv
with open('train_dic.txt','w') as f:
    writer = csv.writer(f)
    for line in train:
        writer.writerow(line)
        
with open('test_dic.txt','w') as f:
    writer = csv.writer(f)
    for line in test:
        writer.writerow(line)
#with open('dic.txt','w') as f:
#    writer = csv.writer(f)
#    writer.writerows(dic)
#np.savetxt('train_dic.txt',train)
#np.savetxt('test_dic.txt',test)

In [382]:
dic = np.asarray(dic,dtype = 'str')
dic.shape

(4361,)

In [384]:
np.savetxt('dic.txt',dic,fmt="%s")
np.savetxt('train_dic.txt',train,fmt="%s")
np.savetxt('test_dic.txt',test,fmt="%s")